# Example of the SASRec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [ ]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback, 
    TorchPredictionCallback
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasRecPredictionDataset,
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionBatch
)

import pandas as pd

## Getting a spark session

In [ ]:
spark_session = get_spark_session()

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [ ]:
!pip install rs-datasets

In [4]:
from rs_datasets import MovieLens

In [5]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [6]:
interactions.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
user_features.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
item_features.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [9]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()
interactions

,user_id,item_id,rating,timestamp
1000138,6040,858,4,0
1000153,6040,2384,4,1
999873,6040,593,5,2
1000007,6040,1961,4,3
1000192,6040,2019,5,4
...,...,...,...,...
825793,4958,2399,1,446
825438,4958,1407,5,447
825724,4958,3264,4,448
825731,4958,2634,3,449


### Split interactions into the train, validation and test datasets using LastNSplitter

In [10]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [11]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [12]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [13]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [14]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [15]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        cardinality=train_dataset.item_count,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
    )
)

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [ ]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [ ]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [ ]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

## Train model
### Create SASRec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [ ]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 512
NUM_WORKERS = 4

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)
checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasRecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

The path to the best model is saved inside checkpoint_callback

In [20]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

## Inference stage
### Prepare Dataloader and logger

In [21]:
prediction_dataloader = DataLoader(
    dataset=SasRecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")

### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback
You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset
You don't need to use all three callbacks. This is shown only for example

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)


trainer = L.Trainer(
    callbacks=[spark_prediction_callback, pandas_prediction_callback, torch_prediction_callback], 
    logger=csv_logger, 
    inference_mode=True
)
trainer.predict(model, dataloaders=prediction_dataloader)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()

In [23]:
spark_res.show()

+-------+-------+------------------+
|user_id|item_id|             score|
+-------+-------+------------------+
|      0|   1642| 8.630412101745605|
|      0|    360| 8.227091789245605|
|      0|     47| 7.907040119171143|
|      0|   2020|7.7642011642456055|
|      0|   2012| 7.738611221313477|
|      0|   2325| 7.504201412200928|
|      0|     12| 7.422617435455322|
|      0|   2016|  7.37091588973999|
|      0|   1020| 7.297569274902344|
|      0|   1019| 7.280248641967773|
|      0|    612| 7.214434623718262|
|      0|   2014| 6.986264705657959|
|      0|   2018| 6.930776596069336|
|      0|   2068|   6.9286789894104|
|      0|   2027| 6.925469875335693|
|      0|   2009| 6.896538257598877|
|      0|   1050|  6.86756706237793|
|      0|    700|  6.81914758682251|
|      0|   1012| 6.815555572509766|
|      0|   2073| 6.714155197143555|
+-------+-------+------------------+
only showing top 20 rows



In [24]:
pandas_res

,user_id,item_id,score
0,0,1642,8.630412
0,0,360,8.227092
0,0,47,7.90704
0,0,2020,7.764201
0,0,2012,7.738611
...,...,...,...
6039,6039,1384,3.058385
6039,6039,1814,3.050523
6039,6039,3107,3.044481
6039,6039,3248,3.01809


In [25]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

tensor(0) tensor([1642,  360,   47, 2020, 2012, 2325,   12, 2016, 1020, 1019,  612, 2014,
        2018, 2068, 2027, 2009, 1050,  700, 1012, 2073, 1743, 1382,  310, 2965,
        3088, 3090, 1346, 2021,  667,  105, 2072, 1979, 2315, 2011,  547, 2285,
        2023,  582, 2985, 1990, 1850,  358, 2807, 1010,   33, 1017, 1459, 2731,
        3546,  974, 2918, 1011, 2030, 2070,  314, 2013, 1058, 2830,  259, 1851,
         554,    1, 1002,  592, 3144, 1964, 3218, 2047, 2631, 1262,  156,  236,
        3414, 2640,  791, 1547, 2692,   59, 3327, 1664,  826, 1018, 1977, 3328,
        1936, 2497, 2033,  999, 1004, 2092, 1000, 1008, 3682, 3676, 3329,  817,
        2054, 3331, 3690, 2788]) tensor([8.6304, 8.2271, 7.9070, 7.7642, 7.7386, 7.5042, 7.4226, 7.3709, 7.2976,
        7.2802, 7.2144, 6.9863, 6.9308, 6.9287, 6.9255, 6.8965, 6.8676, 6.8191,
        6.8156, 6.7142, 6.7059, 6.7041, 6.6634, 6.6354, 6.6311, 6.5398, 6.5328,
        6.5094, 6.4659, 6.4487, 6.4299, 6.2397, 6.2349, 6.2121, 6.1950, 6.162

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [26]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [27]:
recommendations.show()

+------------------+-------+-------+
|             score|user_id|item_id|
+------------------+-------+-------+
| 8.630412101745605|      1|   1688|
| 8.227091789245605|      1|    364|
| 7.907040119171143|      1|     48|
|7.7642011642456055|      1|   2089|
| 7.738611221313477|      1|   2081|
| 7.504201412200928|      1|   2394|
| 7.422617435455322|      1|     13|
|  7.37091588973999|      1|   2085|
| 7.297569274902344|      1|   1033|
| 7.280248641967773|      1|   1032|
| 7.214434623718262|      1|    616|
| 6.986264705657959|      1|   2083|
| 6.930776596069336|      1|   2087|
|   6.9286789894104|      1|   2137|
| 6.925469875335693|      1|   2096|
| 6.896538257598877|      1|   2078|
|  6.86756706237793|      1|   1064|
|  6.81914758682251|      1|    709|
| 6.815555572509766|      1|   1025|
| 6.714155197143555|      1|   2142|
+------------------+-------+-------+
only showing top 20 rows



### Calculating metrics

In [28]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [29]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [30]:
metrics_to_df(result_metrics)

k,1,10,20,100
HitRate,0.065894,0.284437,0.399503,0.675331
MAP,0.065894,0.121790,0.129791,0.136627
MRR,0.065894,0.121790,0.129791,0.136627
NDCG,0.065894,0.159747,0.188858,0.239564
Precision,0.065894,0.028444,0.019975,0.006753
Recall,0.065894,0.284437,0.399503,0.675331


## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

In [31]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

### Wrapping created tensors in the SasRecPredictionBatch entity

In [33]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()}
)

### Run predict step of the SasRec and get scores from the model

In [34]:
with torch.no_grad():
    scores = model.predict_step(batch, 0)
scores

tensor([[ 2.5791,  0.9246,  0.1922,  ..., -1.6555, -2.3930,  1.5935]])

### Getting three items with the highest score

In [35]:
torch.topk(scores, k=3).indices

tensor([[  5, 476, 108]])